In [1]:
!pip install transformers gradio torch --quiet

In [6]:
# # Step 1: Import Required Libraries

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import gradio as gr

In [3]:
# Step 2: Load Pre-trained GPT-2 Model & Tokenizer
# ============================================
# You can try "gpt2", "gpt2-medium", or "gpt2-large" depending on system capacity
model_name = "gpt2-medium"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [4]:
# Step 3: Define Summarization Function
# ============================================
def summarize_text(text):
    """
    This function takes input text, encodes it using GPT-2 tokenizer,
    and generates a summarized version using beam search decoding.
    """
    # Add summarization prompt (helps guide GPT-2)
    prompt = "Summarize this text:\n" + text + "\nSummary:"

    # Encode input text
    inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = inputs.to(device)

    # Generate summary using the GPT-2 model
    summary_ids = model.generate(
        inputs,
        max_length=120,       # maximum length of output
        min_length=40,        # minimum length of output
        num_beams=5,          # beam search for better results
        no_repeat_ngram_size=2,
        temperature=0.7,      # control creativity
        top_p=0.9,
        top_k=50,
        early_stopping=True
    )

    # Decode the generated text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Extract the summary part (after "Summary:")
    if "Summary:" in summary:
        summary = summary.split("Summary:")[-1].strip()

    return summary

In [5]:
# Step 4: Create Gradio Interface
# ============================================
demo = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=6, placeholder="Enter text to summarize...", label="Input Text"),
    outputs=gr.Textbox(label="Generated Summary"),
    title="GPT-2 Deep Learning Text Summarizer",
    description="This model uses a GPT-2 transformer (deep learning model) to generate summaries from input text."
)

# Launch the app
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dbf1115b60ef887e58.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
